# CITS5508 Assignment 1 

- Author: Joo Kai Tay (22489437)

The following notebook contains code for classification tasks involving the Forest type mapping dataset for tasks 1 & 2 and the California Housing Proces dataset in task 3

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Importing useful libraries
import numpy as np
import os
import copy

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 1 Binary Classification

Read the contents of training.csv and testing.csv. It is assumed that both datasets are present in the root directory of this project

In [ ]:
import pandas as pd

testing_data = pd.read_csv('testing.csv')
training_data = pd.read_csv('training.csv')

## 1.1 Display and visualise the dataset

The following two cells display the first rows of both the training and testing dataset. It can be seen by inspection that there are 28 columns that are part of this dataset which can be grouped into 4 distinct categories:
- class: s, d, o, h
    - These describe the 4 classes that make up this dataset
- features b1 - b9
- features that start with pred_minus_obs_H
- features that start with pred_minus_obs_S

In [ ]:
# Displaying the first 5 rows of the dataset
training_data.head()

In [ ]:
# Displaying the first 5 rows of the dataset
testing_data.head()

## 1.2 Simpifying the classification task

The following cell removes all columns whose names begin with pred_minus_obs. This leaves only features b1-b9 in the training and testing sets.

In [ ]:
features = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
training_data = training_data[['class'] + features]
testing_data = testing_data[['class'] + features]

## 1.3 Visualizing the dataset

The following cells display the different features b1-b9 for the training dataset. The following observations can be gleaned from the plots:

- Features b1, b4 and b7 seem to normally distributed.
- Features b2, b3, b5, b6, b8 and b9 are skewed to the right 

***FIND OUT WHAT SKEWED DATASET REPRESENTS***

In [ ]:
# Plotting the different attributes/columns in the dataset

%matplotlib inline
import matplotlib.pyplot as plt
training_data.hist(bins=50, figsize=(20,15))
save_fig("attribute_histogram_plots_training")
plt.show()

In [ ]:
training_data.describe()

The following cells display the different features b1-b9 for the testing dataset. The observations that can be gleaned from the plots are very similar to that of the training dataset:

- Features b1, b4 and b7 seem to normally distributed.
- Features b2, b3, b5, b6, b8 and b9 are skewed to the right 

***FIND OUT WHAT SKEWED DATASET REPRESENTS***

In [ ]:
# Plotting the different attributes/columns in the dataset

%matplotlib inline
import matplotlib.pyplot as plt
testing_data.hist(bins=50, figsize=(20,15))
save_fig("attribute_histogram_plots_testing")
plt.show()

In [ ]:
testing_data.describe()

## 1.4 Counting class instances 

The following code counts the number of instances for each class label in order to determine if we have a balanced dataset. We can view the output of the count_instances() function to determine the number of non-null instances in the dataset.

It can be observed that the training data has an abundance of values for the s and d class label but a significantly lower proportion for values in the o and h class label.

- 136 instances for class 's'
- 105 instances for class 'd'
- 46 instances for class 'o'
- 38 instances for class 'h'

In [ ]:
# Count the occurance of each label class and plot histogram

instances = training_data['class'].value_counts()
class_graph = instances.plot.bar(edgecolor='black', alpha=0.8)
plt.xticks(rotation = 360)

for rect in class_graph.patches:
    height = rect.get_height()
    x_position = rect.get_x() + rect.get_width() / 2
    label = f"{height:.0f}"  # Format the label to be displayed
    class_graph.annotate(label, (x_position, height), textcoords="offset points", xytext=(0, 4), ha='center', fontsize=10)

plt.show()

## 2.5 Feature Scaling

The following code performs appropirate feature scaling on the datasets before doing the classification. The StandardScaler function from the sklearn.preprocessing package is used for this purpose. Standardization scales the data to have a mean value of 0 and standard deviation of 1. This makes it more useful for the classification tasks that we are about to attempt.

The function pre_process_data() returns a new copy of the training and testing dataset with the appropirate features/classes removed.

In [ ]:
# Declaring variables to be passed to pre_process_data()

features_all = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9'] # to be used for all 9 features
classes_all = ['s', 'd', 'o', 'h'] # to be used for all classes
classes_bin = ['s', 'd'] # to be used for classes 's' and 'd'

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Preparing the encoder
enc_X = copy.deepcopy(training_data)
enc_y = enc_X['class']

# s = 0, d = 1, o = 2, h = 3
enc = OrdinalEncoder(categories=[['s','d', 'o', 'h']])
enc.fit(enc_y.values.reshape(-1,1))

In [ ]:
from sklearn.preprocessing import StandardScaler

def pre_process_data(feature_arr, classes_arr):
    # New scaler
    scaler_func = StandardScaler()

    # Preparing training data
    train_X_func = copy.deepcopy(training_data)
    
    # Removes all instances of classes not in classes_arr
    train_X_func = train_X_func[train_X_func['class'].isin(classes_arr)]
    train_y_func = train_X_func['class']
    
    # Removes all but the feature we want
    train_X_func = train_X_func[feature_arr]
    train_X_func = scaler_func.fit_transform(train_X_func.values)
    
    # Preparing testing data
    test_X_func = copy.deepcopy(testing_data)
    # Removes all instances of classes not in classes_arr
    test_X_func = test_X_func[test_X_func['class'].isin(classes_arr)]
    test_y_func = test_X_func['class']
    
    # Removes all but the feature we want
    test_X_func = test_X_func[feature_arr]
    test_X_func = scaler_func.transform(test_X_func.values)
    
    # Encodes y values using ordinal encoder
    train_y_func = enc.transform(train_y_func.values.reshape(-1,1)) 
    test_y_func = enc.transform(test_y_func.values.reshape(-1,1)) 
    
    return train_X_func, train_y_func, test_X_func, test_y_func

## 1.6 Logistic Regression Classifier

This section will perform binary classification using the logistic regression classifier on features b1 and b2. Only example from two classes: 's' and 'd' will be used.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

feature_bin = ['b1']
train_X_b1, train_y_b1, test_X_b1, test_y_b1 = pre_process_data(feature_bin, classes_bin)

# Creating an instance of the logistic regression classifier and fitting it on the training data
log_reg_b1 = LogisticRegression(random_state=42)
log_reg_b1.fit(train_X_b1, train_y_b1.ravel())

# Making predictions on the testing data
y_pred_b1 = log_reg_b1.predict(test_X_b1)

# Evaluation of model performance
accuracy_b1 = accuracy_score(test_y_b1, y_pred_b1)
report_b1 = classification_report(test_y_b1, y_pred_b1)

print("Accuracy of Logistic Regression Classifier on b1:", accuracy_b1)
print("\nReport on Classification:\n", report_b1)

In [ ]:
feature_bin = ['b2']
train_X_b2, train_y_b2, test_X_b2, test_y_b2 = pre_process_data(feature_bin, classes_bin)

# Creating an instance of the logistic regression classifier and fitting it on the training data
log_reg_b2 = LogisticRegression(random_state=42)
log_reg_b2.fit(train_X_b2, train_y_b2.ravel())

# Making predictions on the testing data
y_pred_b2 = log_reg_b2.predict(test_X_b2)

# Evaluation of model performance
accuracy_b2 = accuracy_score(test_y_b2, y_pred_b2)
report_b2 = classification_report(test_y_b2, y_pred_b2)

print("Accuracy of Logistic Regression Classifier on b2:", accuracy_b2)
print("\nReport on Classification:\n", report_b2)

## 1.7 Estimated Probabilities and Decision Boundary

### 1.7.1 Plotting estimated probabilities and decision boundary for logistic regression classifier considering 1 feature

In [ ]:
import random

def plot_graph(feature_bin, X, y, dec_bound, flag):
    plt.figure(figsize=(8, 3))  # extra code – not needed, just formatting
    plt.plot(X, y[:,0], "b--", linewidth=2,
             label="s proba")
    plt.plot(X, y[:,1], "g-", linewidth=2, label="d proba")
    plt.plot([dec_bound, dec_bound], [0, 1], "k:", linewidth=2,
             label="Decision boundary")

    # extra code – this section beautifies and saves Figure 4–23
    plt.arrow(x=dec_bound, y=0.92, dx=0.3, dy=0,
              head_width=0.05, head_length=0.1, fc="b", ec="b")
    plt.arrow(x=dec_bound, y=0.08, dx=-0.3, dy=0,
              head_width=0.05, head_length=0.1, fc="g", ec="g")
    
    if flag == 'Y':
        test_vals_x = np.zeros(shape=(10,1))
        test_vals_y = np.empty(shape=(10,1))
        
        for idx in range(10):
            value = random.randint(0,test_X_b1.size - 1)
            test_vals_x[idx] = np.dot(np.atleast_2d(test_X_all[idx]), coef_matrix.T) + bias
            test_vals_y[idx] = test_y_all[value]
        
        plt.plot(test_vals_x[test_vals_y == 0], test_vals_y[test_vals_y == 0], "g^")
        plt.plot(test_vals_x[test_vals_y == 1], test_vals_y[test_vals_y == 1], "bs")
    
    plt.xlabel(feature_bin)
    plt.ylabel("Probability")
    plt.legend(loc="center right")
    plt.axis([X.min(), X.max(), y.min() - 0.05, y.max() + 0.05])
    plt.grid()
    save_fig("logistic_regression_plot")

    plt.show()

### 1.7.1.1 Feature b1

In [ ]:
#Generate even spaced 1000 datapoints
X_new = np.linspace(train_X_b1.min() - 5, train_X_b1.max() + 5, 1000).reshape(-1,1)
y_proba = log_reg_b1.predict_proba(X_new)

decision_boundary = X_new[y_proba[:, 0] >= 0.5][0, 0]

print('Decision boundary for feature b1:', decision_boundary)

plot_graph('b1', X_new, y_proba, decision_boundary, 'N')

### 1.7.1.2 Feature b2

In [ ]:
#Generate even spaced 1000 datapoints
X_new = np.linspace(train_X_b2.min() - 5, train_X_b2.max() + 5, 1000).reshape(-1,1)
y_proba = log_reg_b2.predict_proba(X_new)

decision_boundary = X_new[y_proba[:, 0] <= 0.5][0, 0]

print('Decision boundary for feature b2:', decision_boundary)

plot_graph('b2', X_new, y_proba, decision_boundary, 'N')

### 1.7.2 Logistic Regression Classifier with all 9 features

In [ ]:
train_X_all, train_y_all, test_X_all, test_y_all = pre_process_data(features_all, classes_bin)

# Creating an instance of the logistic regression classifier and fitting it on the training data
log_reg_all = LogisticRegression(random_state=42)
log_reg_all.fit(train_X_all, train_y_all.ravel())

# Making predictions on the testing data
y_pred_all = log_reg_all.predict(test_X_all)

# Evaluation of model performance
accuracy_all = accuracy_score(test_y_all, y_pred_all)
report_all = classification_report(test_y_all, y_pred_all)

print("Accuracy of Logistic Regression Classifier on all 9 features:", accuracy_all)
print("\nReport on Classification:\n", report_all)

The following cell plots the estimated probabilities and decision boundary for a model considering all features. This is done using the score value of the linear part from the logistic regression model in the x-axis.

10 instances of the testing set are then added to the plot and verfied if the classifier has made a correct decision.

In [ ]:
coef_matrix = log_reg_all.coef_ # Coefficient of the features in the decision function
bias = log_reg_all.intercept_   # Bias term added to the decision function 

num_rows, num_cols = train_X_all.shape
x_axis = np.zeros(shape=(num_rows,1))
y_axis = np.zeros(shape=(num_rows,1))

# Computes the dot product of the training data with the coefficient terms and add the bias term to obtain the score value
for idx in range(num_rows):
    x_axis[idx] = np.dot(np.atleast_2d(train_X_all[idx]), coef_matrix.T) + bias

# Get the probability predictions based on the score value
y_axis = log_reg_all.predict_proba(train_X_all)

# Sort both arrays in ascending order for smooth plotting
x_axis, y_axis = zip(*sorted(zip(x_axis, y_axis)))

decision_boundary_all = np.array(x_axis)[np.array(y_axis)[:, 1] >= 0.5][0, 0]
print('Decision boundary for feature all 9 features:', decision_boundary_all)
plot_graph('Score Value', np.array(x_axis), np.array(y_axis), decision_boundary_all, 'Y')

## 1.8 Precision vs Recall

### 1.8.1 Precision and recall vs threshold

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve

y_scores = cross_val_predict(log_reg_all, train_X_all, train_y_all.ravel(), cv=3,
                             method="decision_function")

precisions, recalls, thresholds = precision_recall_curve(train_y_all, y_scores)
threshold = 1

plt.figure(figsize=(8, 4))  # extra code – it's not needed, just formatting
plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
plt.vlines(threshold, 0, 1.0, "k", "dotted", label="threshold")

# extra code – this section just beautifies and saves Figure 3–5
idx = (thresholds >= threshold).argmax()  # first index ≥ threshold
plt.plot(thresholds[idx], precisions[idx], "bo")
plt.plot(thresholds[idx], recalls[idx], "go")
plt.axis([-8, 5, 0, 1.2])
plt.grid()
plt.xlabel("Threshold")
plt.legend(loc="center right")
save_fig("precision_recall_vs_threshold_plot")

plt.show()

### 1.8.2 Precision vs recall

In [ ]:
import matplotlib.patches as patches  # extra code – for the curved arrow

plt.figure(figsize=(6, 5))  # extra code – not needed, just formatting

plt.plot(recalls, precisions, linewidth=2, label="Precision/Recall curve")

# extra code – just beautifies and saves Figure 3–6
plt.plot([recalls[idx], recalls[idx]], [0., precisions[idx]], "k:")
plt.plot([0.0, recalls[idx]], [precisions[idx], precisions[idx]], "k:")
plt.plot([recalls[idx]], [precisions[idx]], "ko",
         label="Point at threshold 3,000")
plt.gca().add_patch(patches.FancyArrowPatch(
    (0.79, 0.60), (0.61, 0.78),
    connectionstyle="arc3,rad=.2",
    arrowstyle="Simple, tail_width=1.5, head_width=8, head_length=10",
    color="#444444"))
plt.text(0.56, 0.62, "Higher\nthreshold", color="#333333")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.axis([0.4, 1.1, 0.5, 1.0])
plt.grid()
plt.legend(loc="lower left")
save_fig("precision_vs_recall_plot")

plt.show()

## 1.9 K-Nearest-Neighbour (kNN) Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

train_X_knn, train_y_knn, test_X_knn, test_y_knn = pre_process_data(features_all, classes_bin)

kNN = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto', p=1)
kNN.fit(train_X_knn, train_y_knn.ravel())

# Making predictions on the testing data
y_pred_knn = kNN.predict(test_X_knn)

# Evaluation of model performance
accuracy_knn = accuracy_score(test_y_knn, y_pred_knn)
report_knn = classification_report(test_y_knn, y_pred_knn)

print("Accuracy of Logistic Regression Classifier on all 9 features:", accuracy_knn)
print("\nReport on Classification:\n", report_knn)

## 1.10 Comparing Logistic Regression and kNN

In [ ]:
from sklearn.metrics import roc_curve

def plot_roc(y_stuff, y_scores_stuff, label=None):
    fpr, tpr, threshold = roc_curve(y_stuff, y_scores_stuff.ravel())

    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")

    # extra code – just beautifies and saves Figure 3–7
    plt.xlabel('False Positive Rate (Fall-Out)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.grid()
    plt.axis([0, 1, 0, 1.1])
    plt.legend(loc="lower right", fontsize=13)
    save_fig("roc_curve_plot")
    plt.grid(True)      

In [ ]:
y_scores_log = cross_val_predict(log_reg_all, test_X_all, test_y_all.ravel(), cv=3)
y_scores_knn = cross_val_predict(kNN, test_X_knn, test_y_knn.ravel(), cv=3)

fig_roc = plt.figure(figsize=(12,9))

plt.subplot(2,2,1)
plot_roc(test_y_all, y_scores_log)
plt.title('ROC - Logistic Regression All Features')

plt.subplot(2,2,2)
plot_roc(test_y_knn, y_scores_knn)
plt.title('ROC - kNN All Features')

plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

plt.rc('font', size=9)  
ConfusionMatrixDisplay.from_predictions(enc.inverse_transform(test_y_all), enc.inverse_transform(y_scores_log.reshape(-1,1)))
plt.title('Confusion Matrix - Logistic Regression')

plt.rc('font', size=9)  # extra code – make the text smaller
ConfusionMatrixDisplay.from_predictions(enc.inverse_transform(test_y_knn), enc.inverse_transform(y_scores_knn.reshape(-1,1)))
plt.title('Confusion Matrix - kNN')

plt.show()

## 1.11 Cross Validation Scores

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform 3-fold cross-validation 
cv_scores_log_reg = cross_val_score(log_reg_all, test_X_all, test_y_all.ravel(), cv=3)
cv_scores_knn = cross_val_score(kNN, test_X_knn, test_y_knn.ravel(), cv=3)

# Print Array of scores of the estimator for each run of the cross validation.
print("Logistic Regression CV scores:", cv_scores_log_reg)
print("k-NN CV scores:", cv_scores_knn)

# Print average cross-validation scores
print("Logistic Regression average CV score:", cv_scores_log_reg.mean())
print("k-NN average CV score:", cv_scores_knn.mean())

# 2 Multiclass Classification

The following steps will be performed with all classes in the Forest type mapping Data Set

## 2.1 Support Vector Machine Classifier 

Use the Support Vector Machine Classifier implemented in the sklearn.svm.SVC class to perform multiclass classification using the one-versus-one strategy. 

Use grid search and 3-fold cross validation to find the optimal values for the two hyper parameters 

Hyper parameters:
- C (regularization parameter)
- kernel (linear, poly, rbf, sigmoid, precomputed)
- degree (only for poly kernel)
- gamma (rbf, poly, sigmoid)
- coef0 (poly and sigmoid)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Getting new instances of the training and testing set
train_X_svc, train_y_svc, test_X_svc, test_y_svc = pre_process_data(features_all, classes_all)

# Create an SVM classifier using the hyperparameter decision_function_shape='ovo' for a one-versus-one strategy
svc = SVC(decision_function_shape='ovo')

# Define the hyperparameters and their possible values for grid search
param_grid = {
    'C': [0.1, 5, 50],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

# Create a grid search object with 3-fold cross-validation
grid_search = GridSearchCV(svc, param_grid, cv=3, scoring='accuracy')

# Fit the grid search object to the training data
grid_search.fit(train_X_svc, np.ravel(train_y_svc))


# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Print the best cross-validation score
print("Best cross-validation score:", grid_search.best_score_)

# Evaluate the classifier with the best hyperparameters on the test set
best_svc = grid_search.best_estimator_
test_accuracy = best_svc.score(train_X_svc, np.ravel(train_y_svc))
print("Test set accuracy:", test_accuracy)

## 2.2 Softmax Regression

In [ ]:
# Getting new instances of the training and testing set
train_X_softmax, train_y_softmax, test_X_softmax, test_y_softmax = pre_process_data(features_all, classes_all)

# Define the hyperparameters and their possible values for grid search
param_grid = {
    'C': [1, 2, 10],
    'solver': ['lbfgs', 'newton-cg']
}

softmax = LogisticRegression(multi_class='multinomial')

# Create a grid search object with 3-fold cross-validation
grid_search = GridSearchCV(softmax, param_grid, cv=3, scoring='accuracy')

# Fit the grid search object to the training data
grid_search.fit(train_X_softmax, np.ravel(train_y_softmax))


# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Print the best cross-validation score
print("Best cross-validation score:", grid_search.best_score_)

# Evaluate the classifier with the best hyperparameters on the test set
best_svc = grid_search.best_estimator_
test_accuracy = best_svc.score(train_X_softmax, np.ravel(train_y_softmax))
print("Test set accuracy:", test_accuracy)

## 2.3 kNN Classifier

In [ ]:
# Getting new instances of the training and testing set
train_X_kNN1, train_y_kNN1, test_X_kNN1, test_y_kNN1 = pre_process_data(features_all, classes_all)

# Define the hyperparameters and their possible values for grid search
param_grid = {
    'n_neighbors': [1, 19, 9],
    'algorithm': ['ball_tree', 'kd_tree', 'brute']
}

kNN1 = KNeighborsClassifier()

# Create a grid search object with 3-fold cross-validation
grid_search = GridSearchCV(kNN1, param_grid, cv=3, scoring='accuracy')

# Fit the grid search object to the training data
grid_search.fit(train_X_kNN1, np.ravel(train_y_kNN1))


# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Print the best cross-validation score
print("Best cross-validation score:", grid_search.best_score_)

# Evaluate the classifier with the best hyperparameters on the test set
best_svc = grid_search.best_estimator_
test_accuracy = best_svc.score(train_X_kNN1, np.ravel(train_y_kNN1))
print("Test set accuracy:", test_accuracy)

# 3 Implementing k-NN algorithm to do regression

The following cells use the class KNeighborsRegressor to perform regression on the California Housing Prices dataset

In [ ]:
from pathlib import Path
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()

In [ ]:
housing.head()

In [ ]:
housing.info()

## 3.1 Cleaning the data

In [ ]:
# Preparing the encoder
enc_cat = OrdinalEncoder()
# Encodes the ocean proximity category
housing['ocean_proximity'] = enc_cat.fit_transform(housing['ocean_proximity'].values.reshape(-1,1))

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

housing_transformed = imputer.fit_transform(housing)

housing = pd.DataFrame(housing_transformed, columns=housing.columns)

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into a training set (80%) and a testing set (20%)
housing_train, housing_test = train_test_split(housing, test_size=0.2, random_state=42)

# Splitting training set into x and y components
housing_train_X = housing_train.drop("median_house_value", axis=1)
housing_train_y = housing_train["median_house_value"].copy()

# Splitting testing set into x and y components
housing_test_X = housing_test.drop("median_house_value", axis=1)
housing_test_y = housing_test["median_house_value"].copy()

In [ ]:
standard_scaler_housing = StandardScaler()

cols = housing.drop("median_house_value", axis=1)

# Fit the standard scaler to the training set
housing_train_X = standard_scaler_housing.fit_transform(housing_train_X)
housing_train_X_scaled = pd.DataFrame(housing_train_X, columns=cols.columns)


# Transform the testing set
housing_test_X = standard_scaler_housing.transform(housing_test_X)
housing_test_X_scaled = pd.DataFrame(housing_test_X, columns=cols.columns)

## 3.2 RMSE of whole set

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

kNN_reg = KNeighborsRegressor(n_neighbors=5)
kNN_reg.fit(housing_train_X, housing_train_y)

# Make predictions on the test set
y_pred = kNN_reg.predict(housing_test_X)

# Compute the RMSE
rmse = np.sqrt(mean_squared_error(housing_test_y, y_pred))
print("RMSE:", rmse)

## 3.3 Feature Selection

Feature importance scores are used to determine the relativve importance of each feature in the housing dataset. The following cells will show two methods: linear regression and random forest to obtain the feature importance. The features that show up as the most important will be selected to train the KNeighborsRegressor.

### 3.3.1 Feature Selection with linear regression

In [ ]:
selector_log_reg = LogisticRegression()
selector_log_reg.fit(housing_train_X, housing_train_y)

importances = pd.DataFrame(data={
    'Attribute': housing_train_X_scaled.columns,
    'Importance': selector_log_reg.coef_[0]
})
importances = importances.sort_values(by='Importance', ascending=False)

plt.bar(x=importances['Attribute'], height=importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

### 3.3.2 Feature Selection with random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

selector_random_forest = RandomForestRegressor()
selector_random_forest.fit(housing_train_X, housing_train_y)

importances = pd.DataFrame(data={
    'Attribute': housing_train_X_scaled.columns,
    'Importance': selector_random_forest.feature_importances_
})
importances = importances.sort_values(by='Importance', ascending=False)

plt.bar(x=importances['Attribute'], height=importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

## 3.4 k Neighbors Regressor with selected data

The features ranked by importance from the logistic regressor:
- Total rooms
- Median Income
- Latitude
- Longitude
- Population
- Households
- Total bedrooms
- Housing median age
- Ocean proximity

The features ranked by importance from the random forest regressor:
- Median Income
- Ocean proximity
- Latitude
- Longitude
- Housing median age
- Population
- Total rooms
- Total bedrooms
- Households

Based on this data, 6 features were chosen to train the model:
- Median Income
- Ocean proximity
- Latitude
- Longitude
- Total rooms
- Population


In [ ]:
features_kNN_reg = ['median_income', 'longitude', 'latitude', 'ocean_proximity', 'population', 'total_rooms']
train_X_selected = housing_train_X[features_kNN_reg]
test_X_selected = housing_test_X[features_kNN_reg]

# Train k-NN regressors with 'uniform' and 'distance' weights
knn_uniform = KNeighborsRegressor(n_neighbors=5, weights='uniform')
knn_distance = KNeighborsRegressor(n_neighbors=5, weights='distance')

knn_uniform.fit(train_X_selected, housing_train_y)
knn_distance.fit(train_X_selected, housing_train_y)

# Make predictions on the test set
y_pred_uniform = knn_uniform.predict(test_X_selected)
y_pred_distance = knn_distance.predict(test_X_selected)

# Compute the RMSE
rmse_uniform = np.sqrt(mean_squared_error(housing_test_y, y_pred_uniform))
print("RMSE with uniform weights is:", rmse)

rmse_distance = np.sqrt(mean_squared_error(housing_test_y, y_pred_distance))
print("RMSE with distance weights is:", rmse)